In [9]:
from pymongo import MongoClient
import time

# Connect to MongoDB
client = MongoClient('mongodb://mongodb:27017/')
db = client['entities_design']

# Drop collections if they already exist
db.customers.drop()
db.subscriptions.drop()
db.customers_denorm.drop()

In [10]:
# Create collections
customers = db['customers']
subscriptions = db['subscriptions']

# Insert sample customers
customers.insert_many([
    {"_id": 1, "name": "John Doe"},
    {"_id": 2, "name": "Jane Smith"}
])

# Insert sample subscriptions
subscriptions.insert_many([
    {"_id": 1, "customer_id": 1, "plan": "Basic", "status": "Active"},
    {"_id": 2, "customer_id": 1, "plan": "Premium", "status": "Inactive"},
    {"_id": 3, "customer_id": 2, "plan": "Basic", "status": "Active"}
])

InsertManyResult([1, 2, 3], acknowledged=True)

In [ ]:
# Fetch data using multiple queries (Normalized Model)
customer = customers.find_one({"_id": 1})
customer_subscriptions = subscriptions.find({"customer_id": 1})
customer_data = {
    "name": customer['name'],
    "subscriptions": list(customer_subscriptions)
}

print("Normalized Model - Customer Data with Subscriptions:")
print(customer_data)

In [ ]:

# Create denormalized collection
customers_denorm = db['customers_denorm']

# Insert sample data with embedded subscriptions
customers_denorm.insert_many([
    {
        "_id": 1, 
        "name": "John Doe",
        "subscriptions": [
            {"plan": "Basic", "status": "Active"},
            {"plan": "Premium", "status": "Inactive"}
        ]
    },
    {
        "_id": 2,
        "name": "Jane Smith",
        "subscriptions": [
            {"plan": "Basic", "status": "Active"}
        ]
    }
])

# Fetch data using single query (Denormalized Model)
customer_data_denorm = customers_denorm.find_one({"_id": 1})

print("Denormalized Model - Customer Data with Subscriptions:")
print(customer_data_denorm)

In [ ]:
# Perform $lookup aggregation to join customers and subscriptions
pipeline = [
    {
        "$lookup": {
            "from": "subscriptions",
            "localField": "_id",
            "foreignField": "customer_id",
            "as": "subscriptions"
        }
    },
    {
        "$match": {
            "_id": 1
        }
    }
]

result = list(customers.aggregate(pipeline))

print("Normalized Model with $lookup Aggregation - Customer Data with Subscriptions:")
print(result)

# Measure performance of $lookup aggregation
start_time = time.time()
result = list(customers.aggregate(pipeline))
lookup_duration = time.time() - start_time

print("Time taken for $lookup aggregation:", lookup_duration, "seconds")

# Measure performance of fetching from denormalized model
start_time = time.time()
customer_data_denorm = customers_denorm.find_one({"_id": 1})
denorm_duration = time.time() - start_time

print("Time taken for fetching from denormalized model:", denorm_duration, "seconds")
